## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_zweiradparkierung**

## Dataset
# **Zweiradparkierung**

## Description

Der Datensatz umfasst den aktuellen Stand (soweit bekannt) der öffentlich zugänglichen Abstellplätze für Velo, Motorräder oder kombinierte Velo/Motorräder-Abstellplätze auf dem Gebiet der Stadt Zürich. Neben der Lage der Abstellplätze enthält der Datensatz weitere Attribute wie Pfostentyp, Kapazität, Gebührenpflicht. Die Daten werden alle zwei Jahre manuell bei einer Begehung erhoben und der Datensatz entsprechend nachgführt. Aber auch zwischendurch werden bei bekannt gewordenen Änderungen oder Rückmeldungen Daten aktualisiert. Die Daten zeigen nicht in allen Fällen das aktuelle und reale Abstellplatzangebot vor Ort: So ist das temporäre Wegfallen von Abstellplätzen wegen Baustellen oder Ähnlichem nicht berücksichtigt. Bei allen Anlagen wird die Kapazität an Abstellplätzen bestmöglich abgeschätzt. Die Daten enthalten keine Informationen darüber, ob ein Abstellplatz frei oder belegt ist. Auch besteht keine Garantie, dass die aufgeführten Abstellplätze zu jedem Zeitpunkt nutzbar und zugänglich sind. Die gemischten Abstellplätze können trotz sorgfältiger Datenaufnahme in der Praxis an Orten sein, deren Zufahrt für Motorräder verboten ist. Sollten aufgeführte Abstellplätze nicht öffentlich zugänglich sein, so werden diese auf Wunsch oder nach Rückmeldung aus der Datentabelle entfernt. Fehlerhafte Attribute werden nach Rückmeldung ebenfalls korrigiert.

**Zweck**: Parkraumplanung, Statistisches Jahrbuch und statistisches Informationsportal, Controlling Masterplan Velo, Mobilitätsmanagement, allgemeine Planungsgrundlage

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**Zweiradparkierung»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/63aec392-63a3-4dbe-9ceb-5ef7efe77e2e).				
			   

**Datenerfassung:**

wird nachgeführt, sobald ein Abstellplatz gebaut wurde

**Datengrundlage:**

AV-Daten

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/63aec392-63a3-4dbe-9ceb-5ef7efe77e2e.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_zweiradparkierung)

https://www.stadt-zuerich.ch/geodaten/download/Zweiradparkierung?format=geojson_link<br>


## Metadata
- **Publisher** `Grundlagen + Strategien, Tiefbauamt, Tiefbau- und Entsorgungsdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Mobilität`
- **Tags** `['abstellplatz', 'fahrrad', 'geodaten', 'geoportal', 'mofa', 'motorrad', 'parkplatz', 'punktdaten', 'stzh', 'vektordaten', 'velo', 'zweiradabstellplatze']`
- **Metadata_created** `2023-11-06T03:13:20.649438`
- **Metadata_modified** `2025-05-26T05:34:02.755644`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Zweiradparkierung?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch